**Keras-MNIST-CNN-Learning-Curve**  

This is a modified version using MNIST data directly from the Keras repository.  
The train test split is 60k/10k which is different from the Kaggle data (42k/28k)  
The example contains a learning curve and confusion matrix.  
Model ensembles will improve the accuracy.    

Training on GPU over 60 epochs  
Test loss: 0.030968  
Test accuracy: 0.9944  

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input"))

# Any results you write to the current directory are saved as output.

['train.csv', '.ipynb_checkpoints', 'test.csv']


In [3]:

"""
@author: Keras doc
https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
and fchollet
https://www.kaggle.com/fchollet/simple-deep-mlp-with-keras/code
"""

'''Trains a simple convnet on the MNIST dataset.
Original scripts gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU. (12x16s=3.2 minutes)
Geforce 1050: 11 seconds per epoch with batch size 128
Geforce 1050:  8 seconds per epoch with batch size 256
Geforce 1050:  6 seconds per epoch with batch size 512
Test loss: 0.023903
Test accuracy: 0.9923
'''

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
import matplotlib.pyplot as plt


In [6]:
batch_size = 128 #128
num_classes = 10
epochs = 60

# input image dimensions
img_rows, img_cols = 28, 28

# Read data
train = pd.read_csv('input/train.csv')
# Use .iloc for integer-based indexing to select the first column (index 0) for labels
train_labels = train.iloc[:, 0].values.astype('int32')
x_train = (train.iloc[:, 1:].values).astype('float32')

test =  pd.read_csv('input/test.csv')
# Use .iloc for integer-based indexing to select the first column (index 0) for labels
test_labels = test.iloc[:, 0].values.astype('int32')
x_test = (test.iloc[:, 1:].values).astype('float32')


# https://www.codesofinterest.com/2017/09/keras-image-data-format.html
# Keras image_data_format():  channels_last
from keras import backend as K
print("\n")
print("Keras image_data_format(): ",K.image_data_format())

# reshape to (42000, 28, 28, 1)
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    # normalize
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print("\n")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print("\n")

'''
Keras image_data_format():  channels_last
x_train shape: (42000, 28, 28, 1)
42000 train samples
28000 test samples
'''



Keras image_data_format():  channels_last
x_train shape: (42000, 28, 28, 1)


42000 train samples
28000 test samples




'\nKeras image_data_format():  channels_last\nx_train shape: (42000, 28, 28, 1)\n42000 train samples\n28000 test samples\n'

In [8]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(train_labels)

y_test = keras.utils.to_categorical(test_labels)


In [9]:
#取第一筆的答案來看看 one-hot encoding
y_train[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
# Built model with convolutional Layer and Dense layer and dropout
model = Sequential()
model.add(Conv2D(32, kernel_size=(8, 8),
                 activation='relu',
                 input_shape=input_shape))

model.add(Conv2D(64, (8, 8), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))
earlystopping=[EarlyStopping(monitor='loss', patience=5, verbose=1, mode='auto')]

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.1,
          callbacks=earlystopping)

Epoch 1/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.1067 - loss: 2.3010 - val_accuracy: 0.2329 - val_loss: 2.2754
Epoch 2/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1739 - loss: 2.2724 - val_accuracy: 0.3512 - val_loss: 2.2395
Epoch 3/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2455 - loss: 2.2406 - val_accuracy: 0.4426 - val_loss: 2.1956
Epoch 4/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3006 - loss: 2.1990 - val_accuracy: 0.5283 - val_loss: 2.1372
Epoch 5/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3697 - loss: 2.1418 - val_accuracy: 0.5800 - val_loss: 2.0604
Epoch 6/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4170 - loss: 2.0702 - val_accuracy: 0.6248 - val_loss: 1.9638
Epoch 7/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4617 - loss: 1.9821 - val_accuracy: 0.6617 - val_loss: 1.8464
Epoch 8/60
296/296 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.4975 - loss: 1.8772 - val_accuracy: 

In [12]:
# prompt: plot the learing curve

import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()


In [13]:
# predict the values
##score = model.evaluate(x_test, y_test, verbose=0)
##y_pred = model.predict(x_test).round()

# predict the values
print("Generating test predictions...")
preds = model.predict(x_test, verbose=0)

def write_preds(preds, fname):
    # Get the predicted labels (highest probability)
    predicted_labels = preds.argmax(axis=1)
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": predicted_labels}).to_csv(fname, index=False, header=True)

write_preds(preds, "keras-cnn.csv")
print("Finished prediction.")

Generating test predictions...
Finished prediction.


In [ ]:
# plot CNN model (pip3 install pydot graphviz)
##from keras.utils import plot_model
##plot_model(model)


In [20]:
y_test[0], y_pred[0], y_pred[0].argmax()

(array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([6.7042281e-05, 1.4218321e-06, 9.9983358e-01, 7.4367840e-06,
        2.0594562e-06, 6.5503959e-06, 3.3186439e-05, 1.7522281e-05,
        2.9700082e-05, 1.6234657e-06], dtype=float32),
 np.int64(2))

In [21]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

cm = confusion_matrix(y_test.argmax(axis=1) ,preds.argmax(axis=1) )
print(cm)

print("\n")
print('Test loss:', round(score[0],6))
print('Test accuracy:', score[1])


875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Test loss: 0.3171727955341339
Test accuracy: 0.9110714197158813
[[2690    4    9    9    1   12   17    7   18    1]
 [   0 3118   27    4    3   12    2    9   20    2]
 [  21   30 2479   28   56    9   48   79   54    7]
 [  13   22   69 2466    3   90    4   48   45   25]
 [   6   12    8    1 2525    0   59    4   19  139]
 [  48   30   17   95   28 2201   31    8   49   18]
 [  35   23   24    5   15   29 2613    0    3    0]
 [  11   47   38    3   14    5    1 2676    9   90]
 [  19  111   48   90   28   44   19   16 2313   56]
 [  26   33   23   42   72   19    4   86   22 2429]]


Test loss: 0.317173
Test accuracy: 0.9110714197158813
